In [1]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))
alpha=2

In [2]:
#最適解を出力するアルゴリズム
import itertools
def dynamic(V,v0):
    n=len(V)
    Dt={}
    truck={}
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S=frozenset(a)
            for v in range(n):#始点
                for w in S:#終点
                    Dt[(S,v,w)]=float('inf')#無限大に初期化
                    if i==1:
                        Dt[(S,v,w)]=t_cost(V[v],V[w])
                        truck[(S,v,w)]=[w]
                    else:
                        Sw=S-set([w])
                        for u in Sw:
                            z=Dt[(Sw,v,u)]+t_cost(V[u],V[w])
                            p=truck[(Sw,v,u)]+[w]
                            if z<Dt[(S,v,w)]:
                                Dt[(S,v,w)]=z
                                truck[(S,v,w)]=p
                                                         
    Dop={}
    operation={}
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S=frozenset(a)
            for v in range(n):#始点
                for w in S:#終点
                    Dop[(S,v,w)]=Dt[(S,v,w)]#初期値
                    operation[(S,v,w)]=truck[(S,v,w)]
                    if i==1:
                        Dop[(S,v,w)]=Dt[(S,v,w)]
                        operation[(S,v,w)]=truck[(S,v,w)]
                    else:
                        Svw=S-set([v])-set([w])
                        for d in Svw:
                            drone_cost=d_cost(V[v],V[d])+d_cost(V[d],V[w])
                            Sd=S-set([d])
                            z=max(drone_cost,Dt[(Sd,v,w)])
                            if z<Dop[(S,v,w)]:
                                Dop[(S,v,w)]=z
                                operation[(S,v,w)]=truck[(Sd,v,w)]+[[d]]    
                         
    D={}
    route={}
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S=frozenset(a)
            for w in S:
                D[(S,w)]=float('inf')
    for i in range(n):
        D[(frozenset({i}),i)]=Dop[(frozenset({i}),v0,i)]
        route[(frozenset({i}),i)]=[operation[(frozenset({i}),v0,i)]]
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            U=set(a)#Vの部分集合
            T=set(range(n))-U#T=V-U
            for j in range(1,len(T)+1):
                for b in itertools.combinations(T,j):
                    Tset=set(b)#Tの部分集合
                    for u in U:
                        for w in range(n):
                            z=D[(frozenset(U),u)]+Dop[(frozenset(Tset|{w}),u,w)]
                            p=route[(frozenset(U),u)]+[operation[(frozenset(Tset|{w}),u,w)]]
                            if z<D[(frozenset(U|Tset|{u,w}),w)]:
                                D[(frozenset(U|Tset|{u,w}),w)]=z
                                route[(frozenset(U|Tset|{u,w}),w)]=p
                                
                                
    path=route[(frozenset(range(n)),v0)]
    dset=set()
    for i in range(len(path)):
        if type(path[i][-1])==list:
            dset.add(path[i][-1][0])  
            
    return D[(frozenset(range(n)),v0)],path,dset

In [3]:
import itertools
def two_opt(ps):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    return res

In [4]:
import heapq
def mst(ps):
    n = len(ps)
    #prim
    mst = {v:[] for v in range(n)} # MST の隣接リスト
    X = set()
    heap = []
    for v in range(1,n):
        heapq.heappush(heap,(t_cost(ps[0],ps[v]),0,v))
    while len(heap)>0:
        (d,u,v)=heapq.heappop(heap)
        if v not in X:
            X.add(v)
            mst[u].append(v)
            mst[v].append(u)
            for w in range(n):
                if w not in X: heapq.heappush(heap,(t_cost(ps[v],ps[w]),v,w))
    #dfs
    res = []
    stack = [0]
    visited = [False]*n
    while len(stack)>0:
        v = stack.pop()
        if visited[v]: continue
        visited[v]=True
        res.append(v)
        for u in mst[v]: stack.append(u)
    return res 

In [5]:
def dp(ps):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    return route[(0,frozenset(range(n)))] 

In [6]:
def t_pathcost(a,b,route):
    pathcost=0
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
    if a>b:
        for i in range(a,len(route)):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [7]:
import itertools
import numpy
def greedy_core(tsp,depot):#depotはindex指定ではない
    label={}
    operation={}
    flyop={}
    driveop={}
    tsptotal=0
    for i in tsp:
        label[i]='simple'
        tsptotal=tsptotal+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    n=len(label)
    tspdtotal=tsptotal
    while 'simple' in label.values():
        ms=[]
        for i in range(n):
            if label[tsp[i]]!='simple' or tsp[i]==depot:
                ms.append(-float('inf'))
            else:
                mfsi=t_cost(V[tsp[i-1]],V[tsp[i]])+t_cost(V[tsp[i]],V[tsp[(i+1)%n]])-max(d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]]),t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]]))
                ms.append(mfsi)
        for i in range(n):
            if label[tsp[i-1]]=='combined' and tsp[(i-1)]!=depot:
                if label[tsp[i]]=='simple':
                    j=i-2
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[(j-1)]]=='combined':
                            pls=t_cost(V[tsp[i-1]],V[tsp[i]])+operation[(tsp[dronenode],tsp[j-1],tsp[i-1])]-max(d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[i]]),driveop[(tsp[dronenode],tsp[j-1],tsp[i-1])]+t_cost(V[tsp[i-1]],V[tsp[i]]))
                            ms.append(pls)
                            break
                        j=j-1  
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        for i in range(n):
            if label[tsp[(i+1)%n]]=='combined' and tsp[(i+1)%n]!=depot:
                if label[tsp[i]]=='simple':
                    j=(i+2)%n
                    while label[tsp[j]]!='combined':
                        if label[tsp[j]]=='drone':
                            dronenode=j
                        if label[tsp[(j+1)%n]]=='combined':
                            prs=t_cost(V[tsp[i]],V[tsp[(i+1)%n]])+operation[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]-max(d_cost(V[tsp[i]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]]),driveop[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]+t_cost(V[tsp[i]],V[tsp[(i+1)%n]]))
                            ms.append(prs)
                            break
                        j=(j+1)%n
                else: ms.append(-float('inf'))
            else: ms.append(-float('inf'))
        i = numpy.argmax(ms)
        if ms[i]>0:
            tspdtotal=tspdtotal-ms[i]
            if i//n==0:#makeflysaving
                label[tsp[i]]='drone'
                label[tsp[i-1]]='combined'
                label[tsp[(i+1)%n]]='combined'
                flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=d_cost(V[tsp[i-1]],V[tsp[i]])+d_cost(V[tsp[i]],V[tsp[(i+1)%n]])
                driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=t_cost(V[tsp[i-1]],V[tsp[(i+1)%n]])
                operation[(tsp[i],tsp[i-1],tsp[(i+1)%n])]=max(flyop[(tsp[i],tsp[i-1],tsp[(i+1)%n])],driveop[(tsp[i],tsp[i-1],tsp[(i+1)%n])])
            elif i//n==1:#pushleftsaving
                label[tsp[i%n]]='combined'
                label[tsp[(i-1)%n]]='truck'
                j=(i-2)%n
                while label[tsp[j]]!='combined':
                    if label[tsp[j]]=='drone':
                        dronenode=j
                    if label[tsp[j-1]]=='combined':
                        flyop[(tsp[dronenode],tsp[j-1],tsp[i%n])]=d_cost(V[tsp[j-1]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[i%n]])
                        driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])]=driveop[(tsp[dronenode],tsp[j-1],tsp[(i-1)%n])]+t_cost(V[tsp[(i-1)%n]],V[tsp[i%n]])
                        operation[(tsp[dronenode],tsp[j-1],tsp[i%n])]=max(flyop[(tsp[dronenode],tsp[j-1],tsp[i%n])],driveop[(tsp[dronenode],tsp[j-1],tsp[i%n])])
                        break
                    j=j-1
            elif i//n==2:#pushrightsaving
                label[tsp[i%n]]='combined'
                label[tsp[(i+1)%n]]='truck'
                j=(i+2)%n
                while label[tsp[j]]!='combined':
                    if label[tsp[j]]=='drone':
                        dronenode=j
                    if label[tsp[(j+1)%n]]=='combined':
                        flyop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=d_cost(V[tsp[i%n]],V[tsp[dronenode]])+d_cost(V[tsp[dronenode]],V[tsp[(j+1)%n]])
                        driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=driveop[(tsp[dronenode],tsp[(i+1)%n],tsp[(j+1)%n])]+t_cost(V[tsp[i%n]],V[tsp[(i+1)%n]])
                        operation[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])]=max(flyop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])],driveop[(tsp[dronenode],tsp[i%n],tsp[(j+1)%n])])
                        break
                    j=(j+1)%n
        else:
            for l in label:
                if label[l]=='simple':
                    label[l]='combined'

    
    dset=set()
    for i in label:
        if label[i]=='drone':
            dset.add(i)
    return tspdtotal,label,dset
            

In [8]:
def greedy_ph(alg,V,depot):
    tsp=alg(V)
    return greedy_core(tsp,depot)

In [9]:
def MST_gp(V,depot):
    tsp=mst(V)
    return greedy_core(tsp,depot)

In [10]:
def DP_gp(V,depot):
    tsp=dp(V)
    return greedy_core(tsp,depot)

In [11]:
import itertools
def exact_core(tsp,depot):
    T={}
    M={}
    D={}
    Dres=0
    Mdrone={}
    route={}
    n=len(tsp)
    for (start,drone,end) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[drone+1]])+t_pathcost(drone+1,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (end,start,drone) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (drone,end,start) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
        
    for (i,j) in itertools.permutations(range(n),2):
        M[i,j]=float('inf')
        if i<j:
            if i+1==j:
                M[i,j]=t_cost(V[tsp[i]],V[tsp[j]])
            for k in range(i+1,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
        else:
            for k in range(i+1,n):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
            for k in range(0,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
            M[n-1,0]=t_cost(V[tsp[n-1]],V[tsp[0]])

    for i in range(tsp.index(depot)+1,tsp.index(depot)+n+1):
        D[depot,tsp[i%n]]=float('inf')
        Dres=float('inf')
        D[depot,depot]=0
        route[depot,depot]=[[depot]]
        for j in range(tsp.index(depot),i):
            if j%n!=i%n:
                d=M[j%n,i%n]+D[depot,tsp[j%n]]
                r=route[depot,tsp[j%n]]+[[tsp[j%n],tsp[i%n]]]
                if i==tsp.index(depot)+n:
                    if Dres>d:
                        Dres=d
                        route[depot,depot]=r

                else:
                    if D[depot,tsp[i%n]]>d:
                        D[depot,tsp[i%n]]=d
                        route[depot,tsp[i%n]]=r

    pathlen=len(route[depot,depot])
    path=[]
    for i in range(pathlen-1):
        path=path+route[depot,depot][i+1]
        if (route[depot,depot][i+1][0],route[depot,depot][i+1][1]) in Mdrone:
            path=path+[Mdrone[route[depot,depot][i+1][0],route[depot,depot][i+1][1]]]
        else:
            path=path+[[]]
    
    
    dset=set()
    for i in range(len(path)):
        if type(path[i])==list and len(path[i])==1:
            dset.add(path[i][0])

    return Dres,path,dset

In [12]:
import itertools
def exact_ph(alg,V,depot):
    tsp=alg(V)
    return exact_core(tsp,depot)

In [13]:
def MST_ep(V,depot):
    tsp=mst(V)
    return exact_core(tsp,depot)

In [14]:
def DP_ep(V,depot):
    tsp=dp(V)
    return exact_core(tsp,depot)

In [15]:
import itertools
def twopmove(lis,i,j):#iとjの順番を入れ替える
    if i<j: new_list=lis[:i]+[lis[j]]+lis[(i+1):j]+[lis[i]]+lis[(j+1):]
    else: new_list=lis[:j]+[lis[i]]+lis[(j+1):i]+[lis[j]]+lis[(i+1):]
    return new_list
         
def twooptmove(lis,i,j):#2辺を入れ替える
    if i<j: new_list=lis[:i+1]+lis[j:(i+1)%len(lis)-1:-1]+lis[(j+1):]
    else: new_list=[lis[i]]+lis[j::-1]+lis[:(i+1)%len(lis)-1:-1]+lis[j+1:i]
        
    return new_list

def onepmove(lis,i,j):#iをj番目に挿入する
    if i<j: new_list=lis[:i]+lis[(i+1):(j+1)]+[lis[i]]+lis[(j+1):]
    else: new_list=lis[:j]+[lis[i]]+lis[j:i]+lis[(i+1):]

    return new_list
    

In [16]:
def improve(move,heuristic,lis,depot):
    n=len(lis)
    initial=lis
    f=heuristic(initial,depot)[0]
    update=True
    while update:
        update=False
        for (i,j) in itertools.permutations(range(n),2):
            newroute=move(lis,i,j)
            if f>heuristic(newroute,depot)[0]:
                update=True
                initial=newroute
                f=heuristic(newroute,depot)[0]
        if update:
            lis=initial
    return initial

def improve_all(heuristic,lis,depot):                                
    n=len(lis)
    initial=lis
    f=heuristic(initial,depot)[0]
    update=True
    while update:
        update=False
        for (i,j) in itertools.permutations(range(n),2):
            moves=[twopmove(lis,i,j),twooptmove(lis,i,j),onepmove(lis,i,j)]
            for newroute in moves:
                if f>heuristic(newroute,depot)[0]:
                    update=True
                    initial=newroute
                    f=heuristic(newroute,depot)[0]
        if update:
            lis=initial
    return initial
    
    


In [17]:
def improve_twopmove_greedy(alg,V,depot):
    tsp=improve(twopmove,greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_twooptmove_greedy(alg,V,depot):
    tsp=improve(twooptmove,greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_onepmove_greedy(alg,V,depot):
    tsp=improve(onepmove,greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_twopmove_exact(alg,V,depot):
    tsp=improve(twopmove,exact_core,alg(V),depot)
    return exact_core(tsp,depot)

def improve_twooptmove_exact(alg,V,depot):
    tsp=improve(twooptmove,exact_core,alg(V),depot)
    return exact_core(tsp,depot)

def improve_onepmove_exact(alg,V,depot):
    tsp=improve(onepmove,exact_core,alg(V),depot)
    return exact_core(tsp,depot)

In [18]:
def improve_all_greedy(alg,V,depot):
    tsp=improve_all(greedy_core,alg(V),depot)
    return greedy_core(tsp,depot)

def improve_all_exact(alg,V,depot):
    tsp=improve_all(exact_core,alg(V),depot)
    return exact_core(tsp,depot)

In [19]:
def MST_gp_all(V,depot):
    tsp=improve_all(greedy_core,mst(V),depot)
    return greedy_core(tsp,depot)

def MST_ep_all(V,depot):
    tsp=improve_all(exact_core,mst(V),depot)
    return exact_core(tsp,depot)

def DP_gp_all(V,depot):
    tsp=improve_all(greedy_core,dp(V),depot)
    return greedy_core(tsp,depot)

def DP_ep_all(V,depot):
    tsp=improve_all(exact_core,dp(V),depot)
    return exact_core(tsp,depot)

In [20]:
def merge(truck_nodes,drone_nodes,nd,alg,V,depot):
    D={}
    T={}
    t=[V[i] for i in truck_nodes]
    tpath=[truck_nodes[i] for i in alg(t)]
    for drone in drone_nodes:
        for (start,end) in itertools.permutations(tpath,2):
            drone_cost=d_cost(V[start],V[drone])+d_cost(V[drone],V[end])
            truck_cost=t_pathcost(tpath.index(start),tpath.index(end),tpath)
            T[start,drone,end]=max(drone_cost,truck_cost)
    #ドローンとトラックが一緒に移動する場合は考えない
    dc=[b for b in itertools.permutations(drone_nodes,nd)]#ドローンノードの訪れる順番
    tc=[tuple({depot})+a for a in itertools.combinations(set(tpath)-{depot},nd-1)]#トラックノードのうちどれをスタートノード、エンドノードとするか
    for comb in itertools.product(tc,dc):
        d=0
        for j in range(nd):
            d=d+T[comb[0][j],comb[1][j],comb[0][(j+1)%nd]]
        D[comb]=d,tpath
    return min(D.values()),min(D,key=D.get)

In [21]:
def merge_fast(truck_nodes,drone_nodes,nd,alg,V,depot):
    D={}
    T={}
    t=[V[i] for i in truck_nodes]
    tpath=[truck_nodes[i] for i in alg(t)]
    for drone in drone_nodes:
        for (start,end) in itertools.permutations(tpath,2):
            drone_cost=d_cost(V[start],V[drone])+d_cost(V[drone],V[end])
            truck_cost=t_pathcost(tpath.index(start),tpath.index(end),tpath)
            T[start,drone,end]=max(drone_cost,truck_cost)
    #ドローンとトラックが一緒に移動する場合は考えない
    d=[V[i] for i in drone_nodes]
    dpath=[drone_nodes[i] for i in alg(d)]
    dc=[tuple(dpath),tuple(reversed(dpath))]#ドローンノードの訪れる順番
    tc=[tuple({depot})+a for a in itertools.combinations(set(tpath)-{depot},nd-1)]#トラックノードのうちどれをスタートノード、エンドノードとするか
    for comb in itertools.product(tc,dc):
        d=0
        for j in range(nd):
            d=d+T[comb[0][j],comb[1][j],comb[0][(j+1)%nd]]
        D[comb]=d,tpath
    return min(D.values()),min(D,key=D.get)
            
    
            

In [22]:
def shiwake_mst(V,depot,dist):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_nodes=sorted(dist,key=dist.get)[-nd:]
        truck_nodes=list(set(range(n))-set(drone_nodes))   
        D[merge(truck_nodes,drone_nodes,nd,mst,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,mst,V,depot)[0]            
    res=min(D.values())[0]
    path=min(D,key=D.get)
    dset=set(path[1])
    return res,path,dset

In [23]:
def shiwake_mst_fast(V,depot,dist):
    n=len(V)
    ndmin=n//2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_nodes=sorted(dist,key=dist.get)[-nd:]
        truck_nodes=list(set(range(n))-set(drone_nodes))   
        D[merge_fast(truck_nodes,drone_nodes,nd,mst,V,depot)[1]]=merge_fast(truck_nodes,drone_nodes,nd,mst,V,depot)[0]            
    res=min(D.values())[0]
    path=min(D,key=D.get)
    return res,path

In [24]:
def shiwake_dp(V,depot,dist):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_nodes=sorted(dist,key=dist.get)[-nd:]
        truck_nodes=list(set(range(n))-set(drone_nodes))   
        D[merge(truck_nodes,drone_nodes,nd,dp,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,dp,V,depot)[0]            
    res=min(D.values())[0]
    path=min(D,key=D.get)
    dset=set(path[1])
    return res,path,dset

In [25]:
def shiwake_dp_fast(V,depot,dist):
    n=len(V)
    ndmin=n//2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_nodes=sorted(dist,key=dist.get)[-nd:]
        truck_nodes=list(set(range(n))-set(drone_nodes))   
        D[merge_fast(truck_nodes,drone_nodes,nd,dp,V,depot)[1]]=merge_fast(truck_nodes,drone_nodes,nd,dp,V,depot)[0]            
    res=min(D.values())[0]
    path=min(D,key=D.get)
    return res,path

In [26]:
def MST_min(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return shiwake_mst(V,depot,dist)

def MST_sec(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return shiwake_mst(V,depot,dist)

def MST_sum(V,depot):
    n=len(V)
    dist={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return shiwake_mst(V,depot,dist)

def DP_min(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return shiwake_dp(V,depot,dist)

def DP_sec(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return shiwake_dp(V,depot,dist)

def DP_sum(V,depot):
    n=len(V)
    dist={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return shiwake_dp(V,depot,dist)

In [27]:
def MST_min_fast(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return shiwake_mst_fast(V,depot,dist)

def MST_sec_fast(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return shiwake_mst_fast(V,depot,dist)

def MST_sum_fast(V,depot):
    n=len(V)
    dist={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return shiwake_mst_fast(V,depot,dist)

def DP_min_fast(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return shiwake_dp(V,depot,dist)

def DP_sec_fast(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return shiwake_dp(V,depot,dist)

def DP_sum_fast(V,depot):
    n=len(V)
    dist={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return shiwake_dp(V,depot,dist)

In [28]:
def kouho_mst(V,depot,dist,m):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_subnodes=sorted(dist,key=dist.get)[-(nd+m):]
        for s in itertools.combinations(drone_subnodes,nd):
            drone_nodes=list(s)
            truck_nodes=list(set(range(n))-set(drone_nodes))

            D[merge(truck_nodes,drone_nodes,nd,mst,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,mst,V,depot)[0]

    res=min(D.values())[0]
    path=min(D,key=D.get)
    dset=set(path[1])
    return res,path,dset

In [29]:
def kouho_dp(V,depot,dist,m):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_subnodes=sorted(dist,key=dist.get)[-(nd+m):]
        for s in itertools.combinations(drone_subnodes,nd):
            drone_nodes=list(s)
            truck_nodes=list(set(range(n))-set(drone_nodes))

            D[merge(truck_nodes,drone_nodes,nd,dp,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,dp,V,depot)[0]

    res=min(D.values())[0]
    path=min(D,key=D.get)
    dset=set(path[1])
    return res,path,dset

In [30]:
def MST_min_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return kouho_mst(V,depot,dist,2)

def MST_sec_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return kouho_mst(V,depot,dist,2)

def MST_sum_kouho(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_mst(V,depot,wa,2)

def DP_min_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return kouho_dp(V,depot,dist,2)

def DP_sec_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return kouho_dp(V,depot,dist,2)

def DP_sum_kouho(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_dp(V,depot,wa,2)

In [31]:
def DP_sum_kouho4(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_dp(V,depot,wa,4)

def DP_sum_kouho1(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_dp(V,depot,wa,1)

def DP_sum_kouho3(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_dp(V,depot,wa,3)

In [32]:
def MST_sum_kouho4(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_mst(V,depot,wa,4)

def MST_sum_kouho1(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_mst(V,depot,wa,1)

def MST_sum_kouho3(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_mst(V,depot,wa,3)

In [33]:
def MST_dep(V,depot):
    n=len(V)
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}        
    return shiwake_mst(V,depot,dist)

def DP_dep(V,depot):
    n=len(V)
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}        
    return shiwake_dp(V,depot,dist)

def MST_dep_kouho(V,depot):
    n=len(V)
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}
    return kouho_mst(V,depot,dist,2)

def DP_dep_kouho(V,depot):
    n=len(V)
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}
    return kouho_dp(V,depot,dist,2)

In [34]:
import random
import math
import numpy
def mondai_uniform(n):
    V=[(random.uniform(0,100),random.uniform(0,100)) for i in range(n)]
    return V

def mondai_donuts(n):
    V=[]
    for i in range(n):
        angle=random.uniform(0,2*math.pi)
        V.append((25*random.uniform(0.9,1.1)*math.cos(angle)+50,25*random.uniform(0.9,1.1)*math.sin(angle)+50))
    return V

def mondai_donuts_center(n):
    V=[(50,50)]
    for i in range(n-1):
        angle=random.uniform(0,2*math.pi)
        V.append((25*random.uniform(0.9,1.1)*math.cos(angle)+50,25*random.uniform(0.9,1.1)*math.sin(angle)+50))
    return V

def mondai_center(n):
    V=[]
    for i in range(n):
        angle=random.uniform(0,2*math.pi)
        r=numpy.random.normal(0,25)
        V.append((r*math.cos(angle)+50,r*math.sin(angle)+50))
    return V

In [35]:
alg_list=[MST_gp,MST_ep,DP_gp,DP_ep,MST_gp_all,MST_ep_all,DP_gp_all,DP_ep_all,MST_min,MST_sec,MST_sum,DP_min,DP_sec,DP_sum,MST_min_kouho,MST_sec_kouho,MST_sum_kouho,DP_min_kouho,DP_sec_kouho,DP_sum_kouho]


In [36]:
alg_list=[MST_sum,MST_sum_kouho1,MST_sum_kouho,MST_sum_kouho3,MST_sum_kouho4,DP_sum,DP_sum_kouho1,DP_sum_kouho,DP_sum_kouho3,DP_sum_kouho4]

In [33]:
#実行時間
import time
import openpyxl as px
alpha=2
mondai=mondai_chushin
joken='alpha=2,instance=10,problems=100,mondai_chushin,mの=2'
sheet_name='実行時間4'
count=0
alg_times=[[]for i in range(len(alg_list))]
while count<=99:
    print(count)
    V=mondai(10)
    for i in range(len(alg_list)):
        start=time.time()
        alg_list[i](V,0)
        end=time.time()
        alg_times[i].append(end-start)
    count=count+1
    
wb=px.load_workbook('実験結果.xlsx')
wb.create_sheet(sheet_name)
ws=wb[sheet_name]
column_names=['平均実行時間','最大実行時間']
for i in range(len(column_names)):
    ws.cell(row=1, column=i+2, value=column_names[i])
for i in range(len(alg_list)):
    ws.cell(row=i+2, column=1, value=str(alg_list[i]))
ws.cell(row=1,column=1, value=joken)
for i in range(len(alg_list)):
    ws.cell(row=i+2,column=2,value=sum(alg_times[i])/len(alg_times[i]))#average
    ws.cell(row=i+2,column=3,value=max(alg_times[i]))#max
wb.save('実験結果.xlsx')
wb.close()

0


KeyboardInterrupt: 

In [43]:
alg_list=[MST_gp,MST_ep,MST_gp_all,MST_sum_fast]

In [ ]:
#実行時間
import time
import openpyxl as px
alpha=2
mondai=mondai_uniform
joken='alpha=2,mondai_random,instanceの変化,m=2'
sheet_name='実行時間比較'
instance_size=[10,20,30,40,50]
count=0
alg_times=[[[]for j in range(len(instance_size))]for i in range(len(alg_list))]
while count<=9:
    print(count)
    for j in range(len(instance_size)):
        V=mondai(instance_size[j])
        for i in range(len(alg_list)):
            start=time.time()
            alg_list[i](V,0)
            end=time.time()
            alg_times[i][j].append(end-start)
    count=count+1


wb=px.load_workbook('実験結果.xlsx')
wb.create_sheet(sheet_name)
ws=wb[sheet_name]   
for i in range(len(instance_size)):
    ws.cell(row=1, column=i+2, value='instance size={}'.format(instance_size[i]))
for i in range(len(alg_list)):
    ws.cell(row=i+2, column=1, value=str(alg_list[i]))
ws.cell(row=1,column=1, value=joken)
for i in range(len(alg_list)):
    for j in range(len(instance_size)):
        ws.cell(row=i+2,column=j+2,value=sum(alg_times[i][j])/len(alg_times[i][j]))
wb.save('実験結果.xlsx')
wb.close()

0


In [36]:
alg_list=[MST_sum,MST_sum_kouho1,MST_sum_kouho,MST_sum_kouho3,MST_sum_kouho4,DP_sum,DP_sum_kouho1,DP_sum_kouho,DP_sum_kouho3,DP_sum_kouho4]

In [42]:
alpha=2

In [43]:
alg_list=[MST_gp,MST_ep,DP_gp,DP_ep,MST_gp_all,MST_ep_all,DP_gp_all,DP_ep_all,MST_min,MST_sec,MST_sum,DP_min,DP_sec,DP_sum,MST_min_kouho,MST_sec_kouho,MST_sum_kouho,DP_min_kouho,DP_sec_kouho,DP_sum_kouho]


In [44]:
alg_list=[MST_gp,MST_ep,DP_gp,DP_ep,MST_min_fast,MST_sec_fast,MST_sum_fast,DP_min_fast,DP_sec_fast,DP_sum_fast]

In [45]:
#精度
import openpyxl as px
mondai=mondai_uniform
joken='alpha=2,instance=10,problems=100,mondai_uniform,m=2'
sheet_name='a=2,uniform,fast'
count=0
alg_errors=[[] for i in range(len(alg_list))]
while count<=99:
    print(count)
    V=mondai(10)
    opt=round(dynamic(V,0)[0],9)
    for i in range(len(alg_list)):
        obj=round(alg_list[i](V,0)[0],9)
        gosa=((obj-opt)/opt)*100
        alg_errors[i].append(gosa)
    count=count+1
    
    
wb=px.load_workbook('実験結果.xlsx')
wb.create_sheet(sheet_name)
ws=wb[sheet_name]
column_names=['平均誤差','最大誤差','最適値を出力した回数']
for i in range(len(column_names)):
    ws.cell(row=1, column=i+2, value=column_names[i])
for i in range(len(alg_list)):
    ws.cell(row=i+2, column=1, value=str(alg_list[i]))
ws.cell(row=1,column=1, value=joken)
for i in range(len(alg_list)):
    ws.cell(row=i+2,column=2,value=sum(alg_errors[i])/len(alg_errors[i]))
    ws.cell(row=i+2,column=3,value=max(alg_errors[i]))
    ws.cell(row=i+2,column=4,value=alg_errors[i].count(0))
wb.save('実験結果.xlsx')
wb.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [70]:
alg_list=[MST_gp,MST_ep,DP_gp,DP_ep,MST_gp_all,MST_ep_all,DP_gp_all,DP_ep_all,MST_min,MST_sec,MST_sum,DP_min,DP_sec,DP_sum,MST_min_kouho,MST_sec_kouho,MST_sum_kouho,DP_min_kouho,DP_sec_kouho,DP_sum_kouho]


In [71]:
alpha=3

In [72]:
#精度
import openpyxl as px
mondai=mondai_center
joken='instance=10,problems=100,m=2'
sheet_name='drone,a=3,center'
count=0
alg_less=[[] for i in range(len(alg_list))]
alg_great=[[] for i in range(len(alg_list))]
alg_errors=[[] for i in range(len(alg_list))]
while count<=99:
    print(count)
    V=mondai(10)
    opt=dynamic(V,0)[2]
    for i in range(len(alg_list)):
        obj=alg_list[i](V,0)[2]
        alg_less[i].append(len(opt-obj))
        alg_great[i].append(len(obj-opt))
        gosa=len((opt-obj)|(obj-opt))
        alg_errors[i].append(gosa)
    count=count+1

    
wb=px.load_workbook('実験結果.xlsx')
wb.create_sheet(sheet_name)
ws=wb[sheet_name]
column_names=['最適なのにドローンノードにしなかったノード数','最適でないのにドローンノードにしたノードの数','ドローンノードが一致した回数']
for i in range(len(column_names)):
    ws.cell(row=1, column=i+2, value=column_names[i])
for i in range(len(alg_list)):
    ws.cell(row=i+2, column=1, value=str(alg_list[i]))
ws.cell(row=1,column=1, value=joken)
for i in range(len(alg_list)):
    ws.cell(row=i+2,column=2,value=sum(alg_less[i])/len(alg_less[i]))
    ws.cell(row=i+2,column=3,value=sum(alg_great[i])/len(alg_great[i]))
    ws.cell(row=i+2,column=4,value=alg_errors[i].count(0))
wb.save('実験結果.xlsx')
wb.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [36]:
opt={1,2,3}

In [40]:
obj={1,2,4}

In [41]:
len(opt-obj)

1

In [42]:
len(obj-opt)

1